<a href="https://colab.research.google.com/github/ObjectMatrix/google-colab-notebook/blob/main/pinecode_pdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain --upgrade
!pip install pinecone-client
!pip openai 
!pip install pypdf

In [ ]:
!

# PDF Loaders. If unstructured gives you a hard time, try PyPDFLoader
mybook = "https://github.com/ObjectMatrix/google-colab-notebook/blob/main/mml-book.pdf"

from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, PyPDFLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
import json


In [ ]:
loader = PyPDFLoader("./mml-book.pdf")

## Other options for loaders 
# loader = UnstructuredPDFLoader("../data/field-guide-to-data-science.pdf")
# loader = OnlinePDFLoader("https://wolfpaulus.com/wp-content/uploads/2017/05/field-guide-to-data-science.pdf")


In [ ]:
data = loader.load()

In [ ]:
# Note: If you're using PyPDFLoader then it will split by page for you already
print (f'You have {len(data)} document(s) in your data')
print (f'There are {len(data[30].page_content)} characters in your document')

Chunk your data up into smaller units

In [ ]:
# Note: If we're using PyPDFLoader then we'll be splitting for the 2nd time.
# This is optional, test out on your own data.

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

In [ ]:
print (f'Now you have {len(texts)} documents')

Now, let's create embeddings for our documents to do semantic searach 

In [ ]:
from langchain.vectorstores import Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone



In [ ]:
with open('/content/drive/MyDrive/secrets.json', 'r') as f:
    secrets = json.load(f)

KEY = secrets['SECRET_KEY']
pinecone_env = secrets['pinecone_env']
pinecone = secrets["pinecone"]

# Check to see if there is an environment variable with you API keys, if not, use what you put below
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY', 'KEY')

PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', pinecone)
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', pinecone_env) # You may need to switch with your env